In [62]:
source('utils.r')
source('selinf_functions.r')
source('metrics.r')
source('data_generator.r')
source('cov_matrix.r')

## Testing selection results

Comparing Lasso with Selective Inference and plain Lasso with varying SNR and actual proportion of non-zero coefficients.
Both methods use a fixed $\lambda = ||X^Ty||_{\infty}/2$.

In [63]:
set.seed(1)

n_subjects= 25
n_observations = 4
n = n_subjects * n_observations
p = 200
q = 2
SNR = 4
prop_relevant = 0.2

data <- data_generator(n_subjects, n_observations, p, q, SNR, prop_relevant, rho=0.25)
X <- data$X
subjects <- data$subjects
y <- data$y
beta <- data$beta
sd <- data$sd

# fixing lambda
lambda = max(abs(t(X)%*%y))/2
print(lambda)

[1] 127.7169


## Lasso selection

In [64]:
fixed_form = as.formula(
    paste("y ~ 1 +",paste("X", 1:p, sep="", collapse='+'),sep="")
  )

rnd_form = list(subjects =~ 1)


selFun <- function(y) selFun_fixed_lambda(X, subjects, y, fixed_form, rand_form, lambda)

sel <- selFun(y)
sel_vec <- sel$vec
sel_names <- sel$names

In [65]:
print(metrics(sel_vec,c(TRUE,beta!=0)))

$tpr
[1] 0.2682927

$fdr
[1] 0.4210526



## Adding Selective Inference

In [66]:
# Now we can define the function checking the congruency
# with the original selection
checkFun <- function(yb){

  all(selFun(yb)$vec == sel_vec)

}

sel_form = as.formula(
  paste("y ~ ",paste(sel_names[2:length(sel_names)], collapse='+'), "+ (1|subjects)")
)

final_model = lmer(formula = sel_form, data=data.frame(X, subjects, y))

boundary (singular) fit: see help('isSingular')



In [67]:
# and compute valid p-values conditional on the selection
# (this takes some time and will produce a lot of warnings)
suppressWarnings(res <- mocasin(final_model, this_y = y, conditional = FALSE,
               checkFun = checkFun, nrSamples = 100))

Computing inference for variable (location)  1 

  |======================================================================| 100%


Computing inference for variable (location)  2 

  |======================================================================| 100%


Computing inference for variable (location)  3 

  |======================================================================| 100%


Computing inference for variable (location)  4 

  |======================================================================| 100%


Computing inference for variable (location)  5 

  |======================================================================| 100%


Computing inference for variable (location)  6 

  |======================================================================| 100%


Computing inference for variable (location)  7 

  |======================================================================| 100%


Computing inference for variable (location)  8 

  |===============================

In [83]:
# create a boolean vector for the ones selected with BH-procedure at a certain fdr level

sel_vec2 <- sel_with_selinf(res,sel_vec,0.1)
metrics(sel_vec2,c(1,beta!=0))

$tpr
[1] 0.04878049

$fdr
[1] 0